In [8]:
# Import common libraries
import pandas as pd
import numpy as np
from copy import deepcopy
import importlib


# Import MNE processing
from mne.viz import plot_compare_evokeds
from mne import Epochs, events_from_annotations, set_log_level

# Scikit Learn
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Import StatsModels
import statsmodels.formula.api as smf

# Import Plotting Library
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
from mne_nirs.statistics import run_glm

# Internal Packages
import analysis_tools
import system_file_io
import quality_eval
import dynamic_interval_tools
import glm_analysis
import visualizations

# Reset the cache
importlib.reload(analysis_tools)
importlib.reload(system_file_io)
importlib.reload(dynamic_interval_tools)
importlib.reload(glm_analysis)
importlib.reload(visualizations)

<module 'visualizations' from '/Users/nolanbrady/Desktop/fNIRs-data-pipeline/python/visualizations.py'>

In [9]:
# ------------------------------------------------------------------
# Recurring values that we will allow for users to change in the GUI
# ------------------------------------------------------------------

# Length of the measured interval
interval_length = 15

# Time from trigger point being evaluated
tmin = -1
tmax = 15

# How you would like to rename the numeric triggers from Aurora
trigger_id = {'4': 'Control', '2': 'Neutral', '3': 'Inflam', '1':'Practice'}

# What files would you like to ignore while looping through subjects
ignore = [".DS_Store", "sub-03"]

# File path to the folder containing the BIDS data.
root_dir = '../../LabResearch/IndependentStudy/DataAnalysis'

# Mock data folder to test pulling meta data from folder structure.
path = "../../LabResearch/IndependentStudy/Data"

# Does the study have variable task lengths or does the study follow a block design?
variable_epoch_time = True

# Specifies what columns to use for the GLM constrast. If None, contrast is skipped
# Only two columns can be specified
columns_for_glm_contrast = ['Neutral', 'Inflam']

# Specifies what columns to use for group level analysis.
columns_for_group_analysis = ['Neutral', 'Inflam', 'Control']

In [10]:

def run_group_analysis(path, ignore, tmin, tmax, variable_epoch_time, columns_for_glm_contrast, columns_for_group_analysis):
    # Establish the groups, subject names, and snirf file paths for all subjects
    groups, df = system_file_io.import_data_folder(path, ignore)
    # run the analysis for each group
    glm_dataframes = {}
    for group in groups:
        paths = df[df.eq(group).any(1)]['snirf_path'].to_numpy()

        # Processes all Epochs with proper timestamps and collects it in all_epochs
        # all_data is dict of epoch, condition, raw_haemo, raw_intensity, and f_path for all subs
        all_epochs, all_data = analysis_tools.aggregate_epochs(paths=paths, trigger_id=trigger_id, variable_epoch_time = variable_epoch_time)

        # Process the GLM for all participants in the group into a list of design matrixes
        glm_data = glm_analysis.create_design_matrix(all_data)

        # Converts the glm design matrixes into a dataframe
        df_cha, df_con = glm_analysis.create_glm_df(glm_data, columns_for_glm_contrast)

        # Find the groups significant channels
        significant_channels = analysis_tools.find_significant_channels(df_cha)

        glm_dataframes[group] = {"channels_df": df_cha, "contrasts_df": df_con, "significant_channels": significant_channels}
        
    return glm_dataframes

In [11]:
def run_individual_analysis(path, ignore, tmin, tmax, variable_epoch_time, columns_for_glm_contrast, columns_for_group_analysis):
    groups, df = system_file_io.import_data_folder(path, ignore)
    # run the analysis for each group
    glm_dataframes = {}

    paths = df[df.any(1)]['snirf_path'].to_numpy()

    for id, path in enumerate(paths):
        # Processes all Epochs with proper timestamps and collects it in all_epochs
        # all_data is dict of epoch, condition, raw_haemo, raw_intensity, and f_path for all subs
        all_epochs, all_data = analysis_tools.aggregate_epochs(paths=[path], trigger_id=trigger_id, variable_epoch_time = variable_epoch_time)

        # Process the GLM for all participants in the group into a list of design matrixes
        glm_data = glm_analysis.create_design_matrix(all_data)

        # Converts the glm design matrixes into a dataframe
        df_cha, df_con = glm_analysis.create_glm_df(glm_data, columns_for_glm_contrast)

        # Find the groups significant channels
        significant_channels = analysis_tools.find_significant_channels(df_cha)

        glm_dataframes[f'sub-{id+1}'] = {"channels_df": df_cha, "contrasts_df": df_con, "significant_channels": significant_channels}
        
    return glm_dataframes

In [12]:
# group_data = run_group_analysis(path, ignore, tmin, tmax, variable_epoch_time, columns_for_glm_contrast, columns_for_group_analysis)
# group_data

In [13]:
individual_data = run_individual_analysis(path, ignore, tmin, tmax, variable_epoch_time, columns_for_glm_contrast, columns_for_group_analysis)
individual_data

Loading /Users/nolanbrady/Desktop/fNIRs-data-pipeline/python/../../LabResearch/IndependentStudy/Data/anon/sub-06/nirs/sub-06_task-AnonCom_nirs.snirf
Reading 0 ... 18357  =      0.000 ...  1804.493 secs...
[]
[]
[<Epochs |  1 events (all good), -1.25 - 47.5 sec, baseline -1.25 – 0 sec, ~170 kB, data loaded,
 'Practice': 1>]
[<Epochs |  1 events (all good), -1.25 - 47.5 sec, baseline -1.25 – 0 sec, ~170 kB, data loaded,
 'Practice': 1>]
[<Epochs |  1 events (all good), -1.25 - 47.5 sec, baseline -1.25 – 0 sec, ~170 kB, data loaded,
 'Practice': 1>, <Epochs |  1 events (all good), -1.25 - 47.5 sec, baseline -1.25 – 0 sec, ~170 kB, data loaded,
 'Practice': 1>]
[<Epochs |  1 events (all good), -1.25 - 47.5 sec, baseline -1.25 – 0 sec, ~170 kB, data loaded,
 'Practice': 1>, <Epochs |  1 events (all good), -1.25 - 47.5 sec, baseline -1.25 – 0 sec, ~170 kB, data loaded,
 'Practice': 1>]
[<Epochs |  1 events (all good), -1.25 - 47.5 sec, baseline -1.25 – 0 sec, ~170 kB, data loaded,
 'Practice

{'sub-0': {'channels_df': variable Condition    df           mse   p_value        se         t  \
  0          Control  41.0  1.729710e-11  0.007482  0.000002 -2.813978   
  1           Inflam  41.0  1.729710e-11  0.160376  0.000002 -1.429712   
  2          Neutral  41.0  1.729710e-11  0.176306  0.000002  1.375955   
  3         Practice  41.0  1.729710e-11  0.621018  0.000002  0.498181   
  4         constant  41.0  1.729710e-11  0.539354  0.000001  0.618979   
  ...            ...   ...           ...       ...       ...       ...   
  17707      drift_5  41.0  1.890400e-13  0.948980  0.000005 -0.064381   
  17708      drift_6  41.0  1.890400e-13  0.806279  0.000005  0.246820   
  17709      drift_7  41.0  1.890400e-13  0.934886  0.000005 -0.082202   
  17710      drift_8  41.0  1.890400e-13  0.986121  0.000005  0.017502   
  17711      drift_9  41.0  1.890400e-13  0.794215  0.000005 -0.262546   
  
  variable         theta Source Detector Chroma  Significant    ch_name  
  0        